# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
from macrohet import dataio, tile, visualise
import btrack
print(btrack.__version__)
import os
import json 
from tqdm.auto import tqdm
import numpy as np
from natsort import natsorted
import glob
from skimage import io

scale_factor = 2 #5.04
ndim = 2

0.6.4


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
expt_ID = 'PS0000'
base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'
metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = os.path.join(base_dir, 'acquisition/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
Row Column                                              
3   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      INH          0.04            EC50
    9          WT      BDQ          0.02            EC50
4   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      INH          0.04            EC50
    9          WT      BDQ          0.02            EC50
5   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      INH             2            EC99
    9          WT      BDQ           2.5            EC99
6   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      INH             2            EC99
    9          WT      BDQ           2.5            EC99

### Define row and column of choice

In [4]:
row = 3
column = 5

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [5]:
image_dir = os.path.join(base_dir, 'acquisition/Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane=1,
#                              set_channel=1,
#                              set_time = 66,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()

In [6]:
%%time
images = images.compute().compute()

CPU times: user 1h 9min 15s, sys: 6min 38s, total: 1h 15min 54s
Wall time: 3min 12s


### Load downscaled images for faster browsing

In [26]:
%%time
nemo_base_dir = f'/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/users/dayn/macrohet_nemo/{expt_ID}'
# image_dir = os.path.join('/home/dayn/data/', f'macrohet_images/macrohet_images_ds_2x/{row},{column}')
nemo_image_dir = os.path.join(nemo_base_dir, f'acquisition/macrohet_images_ds_2x/{row},{column}') 
images = np.stack([io.imread(fn) for fn in natsorted(glob.glob(os.path.join(nemo_image_dir, '*.png')))])

CPU times: user 25.9 s, sys: 563 ms, total: 26.4 s
Wall time: 54.2 s


# Load tracks

In [27]:
tracks_fn = os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    tracks = hdf.tracks
    segmentation = hdf.segmentation
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=ndim)

[INFO][2023/12/12 11:00:23 AM] Opening HDF file: /mnt/DATA/macrohet/PS0000/labels/macrohet_seg_model/(3, 5).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/DATA/macrohet/PS0000/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2023/12/12 11:00:23 AM] Loading tracks/obj_type_1
INFO:btrack.io.hdf:Loading tracks/obj_type_1
[INFO][2023/12/12 11:00:23 AM] Loading LBEP/obj_type_1
INFO:btrack.io.hdf:Loading LBEP/obj_type_1
[INFO][2023/12/12 11:00:23 AM] Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
[INFO][2023/12/12 11:00:34 AM] Loading segmentation (75, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (75, 6048, 6048)
[INFO][2023/12/12 11:00:34 AM] Closing HDF file: /mnt/DATA/macrohet/PS0000/labels/macrohet_seg_model/(3, 5).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/DATA/macrohet/PS0000/labels/macrohet_seg_model/(3, 5).h5


### Filter tracks? 
Start with a strict filtering for tracks, then refine as you go on

In [28]:
filter = (50, 70)

In [29]:
filtered_tracks = [t for t in tracks if filter[0]<len(t)<=filter[1]]
print(len(filtered_tracks))

151


In [48]:
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(filtered_tracks, ndim=2)

### Recolour segmentation

In [ ]:
col_segmentation = btrack.utils.update_segmentation(segmentation, tracks, scale = (scale_factor,)*ndim)

# Launch napari image viewer

In [49]:
viewer = napari.Viewer()

viewer.add_image(images, 
#                  channel_axis=1,
#                  name=["macrophage", "mtb"],
#                  colormap=["green",  "magenta"],
# #                  contrast_limits=[[100, 6000], [100, 2000]],
#                  contrast_limits=[[0,450], [0,450]], 
#                  visible = True
                 )
# viewer.add_labels(segmentation, 
# #                   num_colors= 1,
#                   #scale=(10, 1, 1,), 
#                   #color='yellow'
#                   name = 'segmentation'
#                  )
# viewer.add_labels(col_segmentation, 
#                   name = 'recolored segmentation'
#                   #scale=(10, 1, 1,), 
#                   #color='yellow'
#                  )
# viewer.add_tracks(napari_tracks, scale = (1,scale_factor,scale_factor), 
#                   tail_length = 0
# #                     properties=properties, 
# #                     graph=graph, 
# #                     name="Properly downscaled tracks", 
# #                     blending="translucent",
# #                     visible=True,
# # #                     scale = (100,1,1)
#                  )
track_layer = viewer.add_tracks(napari_tracks, scale = (1,scale_factor,scale_factor)
#                     properties=properties, 
#                     graph=graph, 
#                     name="Properly downscaled tracks", 
#                     blending="translucent",
#                     visible=True,
# #                     scale = (100,1,1)
                 )
track_layer.display_id = True

In [50]:
visualise.add_napari_grid_overlay(viewer, N_rows_cols=5)

<Shapes layer 'grid_lines' at 0x7f0b456b34c0>

## Create track performance dictionary

In [33]:
track_performance_dir = f'/mnt/DATA/macrohet/PS0000/upstream_development/tracking/tracking_performance/50<v2<=70/{row},{column}/'
# track_performance_dir = os.path.join('/Users/dayn/Library/CloudStorage/GoogleDrive-nthndy@gmail.com/My Drive/crick_drive/data/macrohet_drive', f'upstream_development/tracking/tracking_performance/{row},{column}/')
# track_performance_dir = os.path.join('/Users/dayn/data/macrohet_mac', f'upstream_development/tracking/tracking_performance/{row},{column}/')
os.makedirs(track_performance_dir, exist_ok=True)
track_performance_dict = dict()

### Or load previous track performance dictionary

In [51]:
# load half finished track_performance_dict
previous_dict_fn = f'/mnt/DATA/macrohet/PS0000/upstream_development/tracking/tracking_performance/50<v2<=70/{row},{column}/({row}, {column})_track_assessment.json'
# Open the JSON file for reading
with open(previous_dict_fn, 'r') as json_file:
    # Load the JSON data into a Python dictionary
    track_performance_dict = json.load(json_file)
track_performance_dict

{'425': True,
 '694': True,
 '403': False,
 '427': True,
 '453': False,
 '217': True,
 '1002': False,
 '1893': False,
 '1986': False,
 '2192': False,
 '401': True,
 '412': False,
 '2499': False,
 '1446': False,
 '1003': True,
 '790': False,
 '1552': False,
 '2862': False,
 '432': True,
 '456': True,
 '461': True,
 '416': True,
 '1892': False,
 '430': True,
 '1226': True,
 '1783': True,
 '429,2818': 'Link',
 '431': True,
 '407': True,
 '2675,404': 'Link',
 '398': True,
 '423': True,
 '424': True,
 '428': False,
 '418': False,
 '411': False,
 '411,1683': 'Link',
 '1058': False,
 '466': True,
 '465': True,
 '397': True,
 '426': True,
 '419': True,
 '1735': True,
 '420': True,
 '421': True,
 '802': True,
 '455': False,
 '406': False,
 '410': True,
 '411,1118': 'Link',
 '1118': True,
 '1114,2130': 'Link',
 '414': True,
 '459': True,
 '2234': True,
 '470': True,
 '437': True,
 '1649,2453': 'Link',
 '2453,3126': 'Link',
 '444': True,
 '865': False,
 '441': True,
 '318': False,
 '2952': False,

### Functions to record ground truth tracks

Napari key bindings that record which tracks are True and which are False

In [55]:
@viewer.bind_key("z", overwrite=True)
def true_track(viewer):
    """
    Marks a track as true based on the cursor position in the Napari viewer.

    Parameters
    ----------
    viewer : napari.viewer.Viewer
        The Napari viewer instance.

    """
    # Scale the coordinates for the tracks layer
    scaled_coords = [viewer.cursor.position[0]] + [coord / scale_factor for coord in viewer.cursor.position[1:]]

    # Use scaled coords to extract track ID under cursor
    cell_ID = viewer.layers['napari_tracks'].get_value(scaled_coords)

    if not cell_ID:
        print('cell ID not found')
    else:
        # Add track label to track_dict
        track_performance_dict[int(cell_ID)] = True

        with open(os.path.join(track_performance_dir, f'{row, column}_track_assessment.json'), "w") as file:
            json.dump(track_performance_dict, file)

        print(f"{cell_ID}:True")

    track = [track for track in tracks if track.ID == cell_ID][0]
    points = [[track.t[i], track.y[i] * scale_factor, track.x[i] * scale_factor] for i in range(len(track))]
    name = 'true tracks'

    try:
        # If the layer exists, add the points
        viewer.layers[name].add(points)
    except:
        # If the layer does not exist, create a new layer
        viewer.add_points(points,
                          size=33,
                          symbol='star',
                          face_color='transparent',
                          edge_color='white',
                          edge_width=0.1,
                          name=name,
                          opacity=1
                          # scale=napari_scale
                          )


@viewer.bind_key("x", overwrite=True)
def false_track(viewer):
    """
    Marks a track as false based on the cursor position in the Napari viewer.

    Parameters
    ----------
    viewer : napari.viewer.Viewer
        The Napari viewer instance.

    """
    # Scale the coordinates for the tracks layer
    scaled_coords = [viewer.cursor.position[0]] + [coord / scale_factor for coord in viewer.cursor.position[1:]]

    # Use scaled coords to extract track ID under cursor
    cell_ID = viewer.layers['napari_tracks'].get_value(scaled_coords)

    if not cell_ID:
        print('cell ID not found')
    else:
        # Add track label to track_dict
        track_performance_dict[int(cell_ID)] = False

        with open(os.path.join(track_performance_dir, f'{row, column}_track_assessment.json'), "w") as file:
            json.dump(track_performance_dict, file)

        print(f"{cell_ID}:False")

    track = [track for track in tracks if track.ID == cell_ID][0]
    points = [[track.t[i], track.y[i] * scale_factor, track.x[i] * scale_factor] for i in range(len(track))]
    name = 'false tracks'

    try:
        # If the layer exists, add the points
        viewer.layers[name].add(points)
    except:
        # If the layer does not exist, create a new layer
        viewer.add_points(points,
                          size=33,
                          symbol='x',
                          face_color='transparent',
                          edge_color='white',
                          edge_width=0.1,
                          name=name,
                          opacity=1
                          # scale=napari_scale
                          )


### Track linking key bindings

In [56]:
@viewer.bind_key("c", overwrite=True)
def link_init_track(viewer):
    """
    Initializes the linking of tracks based on the cursor position in the Napari viewer.

    Parameters
    ----------
    viewer : napari.viewer.Viewer
        The Napari viewer instance.

    """
    # Scale the coordinates for the tracks layer
    scaled_coords = [viewer.cursor.position[0]] + [coord / scale_factor for coord in viewer.cursor.position[1:]]

    # Use scaled coords to extract track ID under cursor
    cell_ID = viewer.layers['napari_tracks'].get_value(scaled_coords)

    track = [track for track in tracks if track.ID == cell_ID][0]
    points = [[track.t[i], track.y[i] * scale_factor, track.x[i] * scale_factor] for i in range(len(track))]
    name = 'to-be linked tracks'

    try:
        # If the layer exists, add the points
        viewer.layers[name].add(points)
    except:
        # If the layer does not exist, create a new layer
        viewer.add_points(points,
                          size=33,
                          symbol='tailed_arrow',
                          face_color='transparent',
                          edge_color='white',
                          edge_width=0.1,
                          name=name,
                          opacity=1
                          # scale=napari_scale
                          )

    global previous_ID
    previous_ID = cell_ID


@viewer.bind_key("v", overwrite=True)
def link_init_track(viewer):
    """
    Initializes the linking of tracks and updates the track performance dictionary based on the cursor position.

    Parameters
    ----------
    viewer : napari.viewer.Viewer
        The Napari viewer instance.

    """
    # Scale the coordinates for the tracks layer
    scaled_coords = [viewer.cursor.position[0]] + [coord / scale_factor for coord in viewer.cursor.position[1:]]

    # Use scaled coords to extract track ID under cursor
    cell_ID = viewer.layers['napari_tracks'].get_value(scaled_coords)

    if not cell_ID:
        print('cell ID not found')
    else:
        # Add track label to track_dict
        track_performance_dict[f'{previous_ID},{cell_ID}'] = 'Link'

        with open(os.path.join(track_performance_dir, f'{row, column}_track_assessment.json'), "w") as file:
            json.dump(track_performance_dict, file)

        print(f"{previous_ID},{cell_ID}:'Link'")

    track = [track for track in tracks if track.ID == cell_ID][0]
    points = [[track.t[i], track.y[i] * scale_factor, track.x[i] * scale_factor] for i in range(len(track))]
    name = 'to-be linked tracks'

    try:
        # If the layer exists, add the points
        viewer.layers[name].add(points)
    except:
        # If the layer does not exist, create a new layer
        viewer.add_points(points,
                          size=33,
                          symbol='tailed_arrow',
                          face_color='transparent',
                          edge_color='white',
                          edge_width=0.1,
                          name=name,
                          opacity=1
                          # scale=napari_scale
                          )

### Add previously annotated tracks

In [52]:
for cell_ID in tqdm(track_performance_dict.keys()):
    if track_performance_dict[cell_ID] == True:
        track = [track for track in tracks if track.ID == int(cell_ID)][0]
        points = [[track.t[i], track.y[i] * scale_factor, track.x[i] * scale_factor] for i in range(len(track))]
        name = 'true tracks'
        try:
            # If it exists, add the points
            viewer.layers[name].add(points)
        except:
            # If it doesn't exist, create a new layer
            viewer.add_points(points, 
                              size=33,
                              symbol='star',
                              face_color='transparent',
                              edge_color='white',
                              edge_width=0.1,
                              name=name,
                              opacity=1
                              # scale=napari_scale
                              )
    elif track_performance_dict[cell_ID] == False:
        track = [track for track in tracks if track.ID == int(cell_ID)][0]
        points = [[track.t[i], track.y[i] * scale_factor, track.x[i] * scale_factor] for i in range(len(track))]
        name = 'false tracks'
        try:
            # If it exists, add the points
            viewer.layers[name].add(points)
        except:
            # If it doesn't exist, create a new layer
            viewer.add_points(points, 
                              size=33,
                              symbol='x',
                              face_color='transparent',
                              edge_color='white',
                              edge_width=0.1,
                              name=name,
                              opacity=1
                              # scale=napari_scale
                              )
    else:
        cell_ID1 = int(cell_ID.split(',')[0])
        track = [track for track in tracks if track.ID == int(cell_ID1)][0]
        points = [[track.t[i], track.y[i] * scale_factor, track.x[i] * scale_factor] for i in range(len(track))]
        name = 'to-be linked tracks'
        try:
            # If it exists, add the points
            viewer.layers[name].add(points)
        except:
            # If it doesn't exist, create a new layer
            viewer.add_points(points, 
                              size=33,
                              symbol='tailed_arrow',
                              face_color='transparent',
                              edge_color='white',
                              edge_width=0.1,
                              name=name,
                              opacity=1
                              # scale=napari_scale
                              )
            
        # Add second linked track
        cell_ID2 = int(cell_ID.split(',')[1])
        track = [track for track in tracks if track.ID == int(cell_ID2)][0]
        points = [[track.t[i], track.y[i] * scale_factor, track.x[i] * scale_factor] for i in range(len(track))]
        name = 'to-be linked tracks'
        try:
            # If it exists, add the points
            viewer.layers[name].add(points)
        except:
            # If it doesn't exist, create a new layer
            viewer.add_points(points, 
                              size=33,
                              symbol='tailed_arrow',
                              face_color='transparent',
                              edge_color='white',
                              edge_width=0.1,
                              name=name,
                              opacity=1
                              # scale=napari_scale
                              )

  0%|          | 0/881 [00:00<?, ?it/s]

# Progress:

In [57]:
print()


1784:True
1555:True
538:True
1315:True
2039:True
476:False


/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divid

/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divid

/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divid

160:True


...

/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divid

/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divid

/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divid

/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divid

/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: invalid value encountered in divide
  alpha = (self.head_length + self.current_time - times) / (
/Users/dayn/opt/anaconda3/envs/aerolite/lib/python3.11/site-packages/napari/layers/tracks/tracks.py:309: RuntimeWarning: divid

In [ ]:
track_performance_dict

In [ ]:
visualise.highlight_cell(327, viewer, tracks, scale_factor=scale_factor)

In [ ]:
# 5% takes 30 mins... 10 hours for one position? 

In [ ]:
del track_performance_dict['752,1952']

# Stats

In [ ]:
output_dir = f'/mnt/DATA/macrohet/upstream_development/tracking/tracking_performance/{row},{column}/ground_truth_tracks'

In [ ]:
len(track_performance_dict)

In [ ]:
track_performance_dict

In [ ]:
correct_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == True]
print(len(correct_track_IDs))

false_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == False]
print(len(false_track_IDs))

linked_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == 'Link']
print(len(linked_track_IDs))

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have defined your track_performance_dict dictionary
correct_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == True]
false_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == False]
linked_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == 'Link']

# Get counts
counts = [len(correct_track_IDs), len(false_track_IDs), len(linked_track_IDs)]

# Prepare data
categories = ['Correct', 'False', 'Linked']
colors = ['green', 'pink', 'yellow']  # PIYG color scheme approximation 

# Plot
plt.figure(figsize=(10, 6))
plt.bar(categories, counts, color=colors)
plt.title('Track ID performance')
plt.xlabel('Categories')
plt.ylabel('Counts')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Set the theme and style
sns.set(font='Nimbus Sans', style='white')

# Assuming you have defined your track_performance_dict dictionary
correct_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == True]
false_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == False]
linked_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == 'Link']

# Get counts
counts = [len(correct_track_IDs), len(false_track_IDs), len(linked_track_IDs)]

# Prepare data
categories = ['Correct', 'False', 'Linked']
colors = ['#98d9b6', '#ffd1dc', '#d3d3d3']  # Adjusted color palette 

# Plot
fig, ax = plt.subplots()

# Define autopct function to move labels into the pie
def autopct_format(values):
    def my_format(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{v:d}'.format(v=val) + ' ({p:.2f}%)'.format(p=pct)
    return my_format

# Draw pie chart
ax.pie(counts, labels=categories, autopct=autopct_format(counts), startangle=90, pctdistance=0.4, colors=colors)

# Draw white circle in the middle for the "donut" style
centre_circle = plt.Circle((0,0),0.750,fc='white')
fig.gca().add_artist(centre_circle)

# Equal aspect ratio ensures that pie is drawn as a circle.
ax.axis('equal')  

# Change the font to Helvetica (default font in Apple Keynote)
plt.rcParams['font.family'] = 'Helvetica'

plt.title('Manual Assessment of Tracking Performance (position [3,5])', pad=20, weight='bold')

# plt.savefig(os.path.join(output_dir, 'track_performance_pie_chart_3,5_ground_truth.png'), 
#             bbox_inches='tight', dpi=314)

plt.show()


In [ ]:
total_len=0
for cell_ID in tqdm(track_performance_dict.keys()):
    if isinstance(cell_ID, str):
        cell_ID1 = int(cell_ID.split(',')[0])
        track_len1 = len([t for t in tracks if t.ID == cell_ID1][0])
        cell_ID2 = int(cell_ID.split(',')[1])
        track_len2 = len([t for t in tracks if t.ID == cell_ID2][0])
        track_len = track_len1+track_len2
    else:
        track_len = len([t for t in tracks if t.ID == cell_ID][0])
    total_len += track_len

In [ ]:
total_len


# Creating heatmap for track quality

Correlate to density? Movement? etc.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Set the theme
sns.set(font='Nimbus Sans', style='white')

correct_x, correct_y = [], []
false_x, false_y = [], []

for cell_ID in track_performance_dict.keys():
    if isinstance(cell_ID, str):
        continue
    track = [t for t in tracks if t.ID == int(cell_ID)][0]
    if track_performance_dict[cell_ID] == True:
        correct_x.extend(track.x)
        correct_y.extend(track.y)
    elif track_performance_dict[cell_ID] == False:
        false_x.extend(track.x)
        false_y.extend(track.y)

# Preparing the grid for the histogram
x_bins = np.linspace(min(min(correct_x), min(false_x)), max(max(correct_x), max(false_x)), 10)
y_bins = np.linspace(min(min(correct_y), min(false_y)), max(max(correct_y), max(false_y)), 10)

# Plotting the heatmap for correctly tracked cells
plt.figure(figsize=(10, 8))
plt.hist2d(correct_x, correct_y, bins=[x_bins, y_bins], cmap='Greens')
plt.colorbar(label='Number of Correctly Tracked Cells')
plt.title('Heatmap of Correctly Tracked Cells')
plt.xlabel('X Position')
plt.ylabel('Y Position')

plt.savefig(os.path.join(output_dir, 'heatmap_correct_tracks_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)
plt.show()

# Plotting the heatmap for falsely tracked cells
plt.figure(figsize=(10, 8))
plt.hist2d(false_x, false_y, bins=[x_bins, y_bins], cmap='Reds')
plt.colorbar(label='Number of Falsely Tracked Cells')
plt.title('Heatmap of Falsely Tracked Cells')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.savefig(os.path.join(output_dir, 'heatmap_false_tracks_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font='Nimbus Sans', style='white')

# Get the first ten frames
frames = range(10)

# Filter tracks for the first ten frames
tracks_first_10_frames = [track for track in tracks if any(track.in_frame(frame) for frame in frames)]

# Get the x and y coordinates for the filtered tracks
x_coords = [coord for track in tracks_first_10_frames for coord in track.x[:10]]
y_coords = [coord for track in tracks_first_10_frames for coord in track.y[:10]]

# Create the density map using plt.hist2d()
heatmap, xedges, yedges, _ = plt.hist2d(x_coords, y_coords, bins=10, cmap='PiYG')
plt.colorbar(label='Density')

plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Density Map of Cell Detections (First 10 Frames)')


plt.savefig(os.path.join(output_dir, 'heatmap_cells_t0-10_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


# Scale the x and y coordinates by 5.04
scaled_false_x = [coord * 5.04 for coord in false_x]
scaled_false_y = [coord * 5.04 for coord in false_y]

scaled_x_coords = [coord * 5.04 for coord in x_coords]
scaled_y_coords = [coord * 5.04 for coord in y_coords]

# Set the figure size for both plots
plt.figure(figsize=(12, 6))

# Plot the KDE density plot of falsely tracked cells
plt.subplot(1, 2, 1)  # Create the first subplot
sns.kdeplot(scaled_false_x, scaled_false_y, cmap='Reds', shade=True, shade_lowest=False)
plt.xlim(0, 1200 * 5.04)
plt.ylim(0, 1200 * 5.04)
plt.xlabel('X Coordinate ')
plt.ylabel('Y Coordinate ')
plt.title('Density of Falsely Tracked Cells')

# Plot the KDE density plot of cell detections (first 10 frames)
plt.subplot(1, 2, 2)  # Create the second subplot
sns.kdeplot(scaled_x_coords, scaled_y_coords, cmap='Blues', shade=True, shade_lowest=False)
plt.xlim(0, 1200 * 5.04)
plt.ylim(0, 1200 * 5.04)
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Density of Cell Detections (First 10 Frames)')

# Share the same x and y labels
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.suptitle('Comparison of Density Plots', fontsize=16, y=1.05)

# Remove the spines
sns.despine()


plt.savefig(os.path.join(output_dir, 'density_false_tracks_cell_detec_0-10_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)
# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Scale the x and y coordinates by 5.04
scaled_false_x = [coord * 5.04 for coord in correct_x]
scaled_false_y = [coord * 5.04 for coord in correct_y]

scaled_x_coords = [coord * 5.04 for coord in x_coords]
scaled_y_coords = [coord * 5.04 for coord in y_coords]

# Set the figure size for both plots
plt.figure(figsize=(12, 6))

# Plot the KDE density plot of falsely tracked cells
plt.subplot(1, 2, 1)  # Create the first subplot
sns.kdeplot(scaled_false_x, scaled_false_y, cmap='Greens', shade=True, shade_lowest=False)
plt.xlim(0, 1200 * 5.04)
plt.ylim(0, 1200 * 5.04)
plt.xlabel('X Coordinate ')
plt.ylabel('Y Coordinate ')
plt.title('Density of Correctly Tracked Cells')

# Plot the KDE density plot of cell detections (first 10 frames)
plt.subplot(1, 2, 2)  # Create the second subplot
sns.kdeplot(scaled_x_coords, scaled_y_coords, cmap='Blues', shade=True, shade_lowest=False)
plt.xlim(0, 1200 * 5.04)
plt.ylim(0, 1200 * 5.04)
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Density of Cell Detections (First 10 Frames)')

# Share the same x and y labels
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.suptitle('Comparison of Density Plots', fontsize=16, y=1.05)

# Remove the spines
sns.despine()

plt.savefig(os.path.join(output_dir, 'density_true_tracks_cell_detec_0-10_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)
# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Lists to store the coordinates
correct_x, correct_y = [], []
infected_x, infected_y = [], []

# Iterate over the tracks and separate the coordinates based on track performance and infection status
for track in tracks:
    if track.ID in track_performance_dict.keys():
        if track_performance_dict[track.ID] == True:
            correct_x.extend(track.x)
            correct_y.extend(track.y)
        if track.properties['Infected'][0] == 1:
            infected_x.extend(track.x)
            infected_y.extend(track.y)

# Set the figure size for both plots
plt.figure(figsize=(12, 6))

# Plot the KDE density plot of correctly tracked cells
plt.subplot(1, 2, 1)  # Create the first subplot
sns.kdeplot(correct_x, correct_y, cmap='Greens', shade=True, shade_lowest=False)
plt.xlim(0, 1200)
plt.ylim(0, 1200)
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Density of Correctly Tracked Cells')

# Plot the KDE density plot of infected cells
plt.subplot(1, 2, 2)  # Create the second subplot
sns.kdeplot(infected_x, infected_y, cmap='Blues', shade=True, shade_lowest=False)
plt.xlim(0, 1200)
plt.ylim(0, 1200)
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Density of Infected Cells')

# Share the same y label
plt.ylabel('Y Coordinate')

# Share the same x and y labels
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.suptitle('Comparison of Density Plots', fontsize=16, y=1.05)

# Remove the spines
sns.despine()


plt.savefig(os.path.join(output_dir, 'density_correct_tracks_infected_cells_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)

# Show the plot
plt.show()


### Comparison of image based density and density of incorrectly tracked cells

In [ ]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images')
first_frame = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane='sum_proj',
#                              set_channel=1,
                             set_time = 0,
#                             input_transforms = [input_transforms]
                            ).compute().compute()

In [ ]:
first_frame.shape

In [ ]:
gfp_t0 = first_frame[0]
rfp_t0 = first_frame[1]

In [ ]:
flipped_gfp = np.flipud(gfp_t0)
flipped_rfp = np.flipud(rfp_t0)

In [ ]:
viewer.add_image(flipped_gfp, 
                 contrast_limits=[100, 6000],
                 )
viewer.add_image(flipped_rfp, 
                 contrast_limits=[100, 2000],
                 )

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# # Scale the x and y coordinates by 5.04
# scaled_x_coords = [coord * 5.04 for coord in x_coords]
# scaled_y_coords = [coord * 5.04 for coord in y_coords]

# Scale the x and y coordinates by 5.04
scaled_false_x = [coord * 5.04 for coord in false_x]
scaled_false_y = [coord * 5.04 for coord in false_y]


# Set the figure size for both plots
plt.figure(figsize=(10, 10))

# Remove axis labels and tick marks
plt.axis('off')

# Load and overlay the image
image = plt.imread("/home/dayn/r03c05f0-p0-ch1sk1fk1fl1.png")
plt.imshow(np.flipud(image), extent=[0, 1200 * 5.04, 0, 1200 * 5.04], alpha=1)

# Plot the KDE density plot of cell detections (first 10 frames)
sns.kdeplot(scaled_false_x, scaled_false_y, cmap='PiYG_r', levels=15, linewidths=3, alpha=0.8)

# Set the image and KDE plot in the center
plt.xlim(0, 1200 * 5.04)
plt.ylim(0, 1200 * 5.04)

# Add a black border with width 50
plt.gca().spines['top'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_color('black')
plt.gca().spines['right'].set_color('black')
plt.gca().spines['top'].set_linewidth(50)
plt.gca().spines['bottom'].set_linewidth(50)
plt.gca().spines['left'].set_linewidth(50)
plt.gca().spines['right'].set_linewidth(50)

# Remove the spines
sns.despine()

# # Save the plot with a black border
# plt.savefig('/home/dayn/density_false_tracks_r03c05.png', bbox_inches='tight', pad_inches=0, transparent=True)

plt.title('Density of false tracks overlaid on first frame of timelapse (position 3,5)')
# Show the plot


plt.savefig(os.path.join(output_dir, 'density_false_tracks_overlay_15lvl_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# # Scale the x and y coordinates by 5.04
# scaled_x_coords = [coord * 5.04 for coord in x_coords]
# scaled_y_coords = [coord * 5.04 for coord in y_coords]

# Scale the x and y coordinates by 5.04
scaled_false_x = [coord * 5.04 for coord in false_x]
scaled_false_y = [coord * 5.04 for coord in false_y]


# Set the figure size for both plots
plt.figure(figsize=(10, 10))

# Remove axis labels and tick marks
plt.axis('off')

# Load and overlay the image
image = plt.imread("/home/dayn/r03c05f0-p0-ch1sk1fk1fl1.png")
plt.imshow(np.flipud(image), extent=[0, 1200 * 5.04, 0, 1200 * 5.04], alpha=1)

# Plot the KDE density plot of cell detections (first 10 frames)
sns.kdeplot(scaled_false_x, scaled_false_y, cmap='PiYG_r', levels=15, linewidths=5, alpha=0.8)

# Set the image and KDE plot in the center
plt.xlim(0, 1200 * 5.04)
plt.ylim(0, 1200 * 5.04)

# Remove the spines
sns.despine()

# # Save the plot with a black border
# plt.savefig('/mnt/DATA/macrohet/upstream_development/tracking/tracking_performance/3,5/ground_truth_tracks/density_false_tracks_r03c05_t01_15lvls.png', 
#             bbox_inches='tight', pad_inches=0, transparent=True)

plt.title('Density of false tracks overlaid on first frame of timelapse (position 3,5)')
# Show the plot

plt.savefig(os.path.join(output_dir, 'density_false_tracks_overlay_15lvl_thicker_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Scale the x and y coordinates by 5.04
scaled_false_x = [coord * 5.04 for coord in false_x]
scaled_false_y = [coord * 5.04 for coord in false_y]

# Set the figure size for both plots
plt.figure(figsize=(10, 10))

# Remove axis labels and tick marks
plt.axis('off')

# Load and overlay the image
image = plt.imread("/home/dayn/r03c05f0-p0-ch1sk1fk1fl1_with_IDs.png")
plt.imshow(np.flipud(image), extent=[0, 1200 * 5.04, 0, 1200 * 5.04], alpha=1)

# Plot the KDE density plot of cell detections (first 10 frames)
sns.kdeplot(scaled_false_x, scaled_false_y, cmap='PiYG_r', levels=15, linewidths=5, alpha=0.666)

# Set the image and KDE plot in the center
plt.xlim(0, 1200 * 5.04)
plt.ylim(0, 1200 * 5.04)

# Invert the y-axis
plt.gca().invert_yaxis()

# Add a black border with width 50
plt.gca().spines['top'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_color('black')
plt.gca().spines['right'].set_color('black')
plt.gca().spines['top'].set_linewidth(50)
plt.gca().spines['bottom'].set_linewidth(50)
plt.gca().spines['left'].set_linewidth(50)
plt.gca().spines['right'].set_linewidth(50)

# Remove the spines
sns.despine()

# # Save the plot with a black border
# plt.savefig('/mnt/DATA/macrohet/upstream_development/tracking/tracking_performance/3,5/ground_truth_tracks/density_false_tracks_r03c05_t01_15lvls_IDs.png', 
#             bbox_inches='tight', pad_inches=0, transparent=True)

plt.title('Density of false tracks overlaid on first frame of timelapse (position 3,5)')


plt.savefig(os.path.join(output_dir, 'density_false_tracks_overlay_15lvl_IDs_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)
# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Scale the x and y coordinates by 5.04
scaled_correct_x = [coord * 5.04 for coord in correct_x]
scaled_correct_y = [coord * 5.04 for coord in correct_y]


# Set the figure size for both plots
plt.figure(figsize=(10, 10))

# Remove axis labels and tick marks
plt.axis('off')

# Load and overlay the image
image = plt.imread("/home/dayn/r03c05f0-p0-ch1sk1fk1fl1_with_IDs.png")
plt.imshow(np.flipud(image), extent=[0, 1200 * 5.04, 0, 1200 * 5.04], alpha=1)

# Plot the KDE density plot of cell detections (first 10 frames)
sns.kdeplot(scaled_correct_x, scaled_correct_y, cmap='PiYG', levels=15, linewidths=5, alpha=0.666)

# Set the image and KDE plot in the center
plt.xlim(0, 1200 * 5.04)
plt.ylim(0, 1200 * 5.04)

# Invert the y-axis
plt.gca().invert_yaxis()

# Add a black border with width 50
plt.gca().spines['top'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_color('black')
plt.gca().spines['right'].set_color('black')
plt.gca().spines['top'].set_linewidth(50)
plt.gca().spines['bottom'].set_linewidth(50)
plt.gca().spines['left'].set_linewidth(50)
plt.gca().spines['right'].set_linewidth(50)

# Remove the spines
sns.despine()

plt.title('Density of correct tracks overlaid on first frame of timelapse (position 3,5)')

plt.savefig(os.path.join(output_dir, 'density_correct_tracks_overlay_15lvl_IDs_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)
# Show the plot
plt.show()


# Creating animation showing true tracks

In [ ]:
true_IDs = [int(ID) for ID, status in track_performance_dict.items() if status is True]
true_tracks = [t for t in tracks if t.ID in true_IDs]

In [ ]:
false_IDs = [int(ID) for ID, status in track_performance_dict.items() if status is False]
false_tracks = [t for t in tracks if t.ID in false_IDs]

In [ ]:
len(true_tracks), len(false_tracks)

In [ ]:
true_napari_tracks, properties, graph = btrack.utils.tracks_to_napari(true_tracks, ndim=ndim)

In [ ]:
viewer.add_tracks(true_napari_tracks, scale = (1,scale_factor,scale_factor))

# Can we characterise the true tracks vs false tracks??

In [ ]:
sns.set(font='Nimbus Sans', style = 'white')

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# Calculate the track lengths for true tracks and false tracks
true_track_lengths = [len(t) for t in true_tracks]
false_track_lengths = [len(t) for t in false_tracks]

# Set the color palette for true and false tracks
colors = sns.color_palette('PiYG')

# Plot the histogram
sns.histplot(true_track_lengths, color=colors[-1], label='True Tracks')
sns.histplot(false_track_lengths, color=colors[0], label='False Tracks', alpha = 0.333)

# Add labels and title
plt.xlabel('Track Length')
plt.ylabel('Count')
plt.title('Distribution of Track Lengths')

# Remove spines and set offset
sns.despine(offset=10)

# Add legend
plt.legend()

plt.savefig(os.path.join(output_dir, 'distribution_true_false_track_lengths_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)

# Show the plot
plt.show()


In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# Calculate the track lengths for true tracks and false tracks
true_track_area_t0 = [t.properties['area'][0] for t in true_tracks]
false_track_area_t0 = [t.properties['area'][0] for t in false_tracks]

# Set the color palette for true and false tracks
colors = sns.color_palette('PiYG')

# Plot the histogram
sns.histplot(true_track_area_t0, color=colors[-1], label='True Tracks')
sns.histplot(false_track_area_t0, color=colors[0], label='False Tracks', alpha = 0.5)

# Add labels and title
plt.xlabel('Cell Area at t = 0')
plt.ylabel('Count')
plt.title('Distribution of t = 0 area between correct and false tracks')

# Remove spines and set offset
sns.despine(offset=10)

# Add legend
plt.legend()

plt.savefig(os.path.join(output_dir, 'distribution_true_false_track_areas_t0_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)

# Show the plot
plt.show()


In [ ]:
true_tracks[0]

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# Calculate the track lengths for true tracks and false tracks
true_track_mtb_t0 = [t.properties['mean_intensity'][2][0] for t in true_tracks]
false_track_mtb_t0 = [t.properties['mean_intensity'][2][0] for t in false_tracks]

# Set the color palette for true and false tracks
colors = sns.color_palette('PiYG')

# Plot the histogram
sns.histplot(true_track_mtb_t0, color=colors[-1], label='True Tracks')
sns.histplot(false_track_mtb_t0, color=colors[0], label='False Tracks', alpha = 0.5)

# Add labels and title
plt.xlabel('Mtb measure at t = 0')
plt.ylabel('Count')
plt.title('Distribution of t = 0 Mtb measure between correct and false tracks')

# Remove spines and set offset
sns.despine(offset=10)

# Add legend
plt.legend()

plt.savefig(os.path.join(output_dir, 'distribution_true_false_track_mtb_t0_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)

# Show the plot
plt.show()


# Truer sense of tracking performance

By counting the number of cells and comparing the number of true tracks to that, rather than just hte ratio of true tracks to false which will include fragmented tracks

In [ ]:
N_cells = 0
for i in range(0, 9):
    N_cells+= np.max(segmentation[i])
N_cells = N_cells/9
print(int(N_cells))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Set the theme and style
sns.set(font='Nimbus Sans', style='white')

# Assuming you have defined your track_performance_dict dictionary
correct_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == True]
false_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == False]
linked_track_IDs = [key for key in track_performance_dict.keys() if track_performance_dict[key] == 'Link']

N_cells = 0
for i in range(0, 9):
    N_cells += np.max(segmentation[i])
N_cells = N_cells / 9

N_false = N_cells - len(correct_track_IDs) - len(linked_track_IDs)

# Get counts
counts = [len(correct_track_IDs), 
          N_false,
          # len(false_track_IDs),
          len(linked_track_IDs)]

# Prepare data
categories = ['Correct', 'False', 'Linked']
colors = ['#98d9b6', '#ffd1dc', '#d3d3d3']  # Adjusted color palette 

# Plot
fig, ax = plt.subplots()

# Define autopct function to move labels into the pie
def autopct_format(values):
    def my_format(pct):
        total = sum(values)
        val = int(round(pct * total / 100.0))
        return '{v:d}'.format(v=val) + ' ({p:.2f}%)'.format(p=pct)
    return my_format

# Draw pie chart
ax.pie(counts, labels=categories, autopct=autopct_format(counts), startangle=90, pctdistance=0.4, colors=colors)

# Draw white circle in the middle for the "donut" style
centre_circle = plt.Circle((0, 0), 0.750, fc='white')
fig.gca().add_artist(centre_circle)

# Equal aspect ratio ensures that pie is drawn as a circle.
ax.axis('equal')  

# Change the font to Helvetica (default font in Apple Keynote)
plt.rcParams['font.family'] = 'Helvetica'

# Add title and subtitle
plt.title('Manual Assessment of Tracking Performance (position 3,5)', pad=20, weight='bold')
sns.set(font='Nimbus Sans', style='white')
plt.text(0, 1.15, 'Fragmented false tracks removed', ha='center', va='center', fontsize=11)

plt.savefig(os.path.join(output_dir, 'true_measure_track_performance_pie_chart_3,5_ground_truth.png'), 
            bbox_inches='tight', dpi=314)

plt.show() 


In [ ]:
output_dir

In [ ]:
temp_v = napari.Viewer()

temp_v.add_image(segmentation, scale=(100,1,1))